In [33]:
import os
import os.path as op
import tensorflow.experimental.numpy as tnp
import numpy as np

In [2]:
pycsou_path = "/Users/hugofluhr/chuv/repositories/pycsou/src/pycsou"

In [3]:
def get_python_files(dir_path):
    python_files = []
    for root, dirs, files in os.walk(pycsou_path):
        for name in files:
            if name.endswith((".py")):
                file_path = op.join(root, name)
                python_files.append(file_path)
    return python_files

def get_files_containing_string(files, string_to_find):
    files_with_string = []
    for file in files:
        with open(file) as f:
            if string_to_find in f.read() :
                files_with_string.append(file)
    return files_with_string

In [4]:
python_files = get_python_files(pycsou_path)

In [5]:
string_to_find = "xp."
files_using_xp = get_files_containing_string(python_files, string_to_find)

print(f"{len(files_using_xp)} files using {string_to_find} out of {len(python_files)} python files")

8 files using xp. out of 21 python files


In [6]:
get_files_containing_string(python_files, "cp.")

['/Users/hugofluhr/chuv/repositories/pycsou/src/pycsou/util/deps.py',
 '/Users/hugofluhr/chuv/repositories/pycsou/src/pycsou/operator/linop/base.py']

In [7]:
files_using_xp

['/Users/hugofluhr/chuv/repositories/pycsou/src/pycsou/abc/operator.py',
 '/Users/hugofluhr/chuv/repositories/pycsou/src/pycsou/util/__init__.py',
 '/Users/hugofluhr/chuv/repositories/pycsou/src/pycsou/_dev/__init__.py',
 '/Users/hugofluhr/chuv/repositories/pycsou/src/pycsou/math/linalg.py',
 '/Users/hugofluhr/chuv/repositories/pycsou/src/pycsou/compound/__init__.py',
 '/Users/hugofluhr/chuv/repositories/pycsou/src/pycsou/opt/stop.py',
 '/Users/hugofluhr/chuv/repositories/pycsou/src/pycsou/opt/solver/cg.py',
 '/Users/hugofluhr/chuv/repositories/pycsou/src/pycsou/operator/linop/base.py']

In [8]:
xp_lines = []
for file in files_using_xp :
    with open(file) as f:
        for num, line in enumerate(f, 1):
            if "xp." in line:
                #print(f"line {num} >>>" + repr(line))
                xp_lines.append(line)


In [9]:
def find_parens(s):
    toret = {}
    pstack = []

    for i, c in enumerate(s):
        if c == '(':
            pstack.append(i)
        elif c == ')':
            if len(pstack) == 0:
                continue
                #raise IndexError("No matching closing parens at: " + str(i))
            toret[pstack.pop()] = i

    #if len(pstack) > 0:
    #    raise IndexError("No matching opening parens at: " + str(pstack.pop()))

    return sorted(toret.items())

In [10]:
def get_usage(line, module='xp.'):
    from_module = line[line.index(module):]
    parent_pairs = find_parens(from_module)
    if not parent_pairs :
        return{"fct":from_module[:from_module.index("\n")], "args":None}
    else :
        function = from_module[:parent_pairs[0][0]]
        args = from_module[parent_pairs[0][0]:parent_pairs[0][1]+1]
        return {"fct":function, "args":args}

In [11]:
xp_usage = [get_usage(line) for line in xp_lines]

In [12]:
xp_usage

[{'fct': 'xp.linalg.norm',
  'args': '(arr - self.prox(arr, tau=mu), axis=-1, keepdims=True)'},
 {'fct': 'xp.eye', 'args': '(self.shape[1], dtype=dtype)'},
 {'fct': 'xp.zeros', 'args': '((*sh_xf[:-1], y0.size), dtype=y0.dtype)'},
 {'fct': 'xp.linalg.norm', 'args': '(arr, axis=-1, keepdims=True)'},
 {'fct': 'xp.linalg.norm', 'args': '(arr, ord=1)'},
 {'fct': 'xp.clip', 'args': '(xp.abs(arr) - tau, a_min=0, a_max=None)'},
 {'fct': 'xp.swapaxes', 'args': '(arr, self.axis, -1)'},
 {'fct': 'xp.zeros_like', 'args': '(arr)'},
 {'fct': 'xp.swapaxes', 'args': '(arr, self.axis, -1)'},
 {'fct': 'xp.zeros_like', 'args': '(arr)'},
 {'fct': 'xp.swapaxes', 'args': '(arr, self.axis, -1)'},
 {'fct': 'xp.zeros_like', 'args': '(arr)'},
 {'fct': 'xp.swapaxes', 'args': '(arr, self.axis, -1)'},
 {'fct': 'xp.zeros_like', 'args': '(arr)'},
 {'fct': 'xp.swapaxes', 'args': '(arr, self.axis, -1)'},
 {'fct': 'xp.zeros_like', 'args': '(arr)'},
 {'fct': 'xp.swapaxes', 'args': '(arr, self.axis, -1)'},
 {'fct': 'xp.z

In [43]:
xp_functions = sorted(list(set([call.get("fct").replace("xp.","").replace("(","") for call in xp_usage])))

In [44]:
xp_functions

['all if self._satisfy_all else any',
 'array',
 'asarray',
 'broadcast_to',
 'clip',
 'concatenate',
 'eye',
 'isnan',
 'linalg',
 'linalg.norm',
 'split',
 'stack',
 'sum',
 'swapaxes',
 'trace',
 'zeros',
 'zeros_like']

In [50]:
for func in xp_functions[1:]:
    try :
        getattr(tnp,func)
    except :
        print(f"No implementation for '{func}'")


No implementation for 'linalg'
No implementation for 'linalg.norm'
